# LDA Algorithm

In [ ]:
# Some IPython magic
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pymc as pm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Preprocessing data

- We take the documents in form of strings, remove the punctuation marks and split the strings in words
- Label the words of the documents by indexing them in order of their appearance 
    + e.g. "My friend is not my enemy" becomes ( 0, 1, 2, 3, 0, 4)


In [ ]:
import string
d1 = "I had a peanuts butter sandwich for breakfast."
d2 = "I like to eat almonds, peanuts and walnuts."
d3 = "My neighbor got a little dog yesterday."
d4 = "Cats and dogs are mortal enemies."
d5 = "You mustn’t feed peanuts to your dog."
docs = [d1, d2, d3, d4, d5]
# create a translation table that will be used to map punctuation characters into empty strings
remove_punctuation = str.maketrans('', '', string.punctuation) 
docs = [doc.translate(remove_punctuation).split() for doc in docs] # remove punctuation marks and split sentences in words

def factorize_docs(docs):
    index = 0
    wdict = {}
    data = []
    for doc in docs:
        doc_data = []
        for word in doc:
            if word not in wdict:
                wdict[word] = index
                index += 1
            doc_data.append(wdict[word])
        data.append(doc_data)
    return np.array(data), wdict

data, wdict = factorize_docs(docs)



## The generative model
- extract the number of words and number of documents from the data
- create the random variables according to the [LDA Algorithm](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation#Model)

In [ ]:
K = 2 # number of topics
V = max(wdict.values()) + 1 # number of words in vocabulary; add one because we start indexing from 0
M = len(data) # number of documents

N = [len(doc) for doc in data] # number of words in each document

#words per topic a priori distribution - V-dimensional
beta = np.ones(V)
phi = pm.Container([pm.CompletedDirichlet("phi_%s" % topic, pm.Dirichlet("pphi_%s" % topic, theta=beta)) for topic in range(K)])

# topics per document a priori distribution - K-dimensional 
alpha = np.ones(K)
theta = pm.Container([pm.CompletedDirichlet("theta_%s" % doc, pm.Dirichlet("ptheta_%s" % doc, theta=alpha)) for doc in range(M)])

# draw topics for each word of each document - a categorical for each word of the doc, based on the document's topic distribution
z = pm.Container([pm.Categorical('z_%i' % doc, 
                     p = theta[doc], 
                     size=N[doc],
                     value=np.random.randint(K, size=N[doc]))
                  for doc in range(M)])

# draw phisical words from word distributions according to each topic
w = pm.Container([pm.Categorical("w_%i_%i" % (doc,word),
                    p = pm.Lambda('phi_z_%i_%i' % (doc,word), 
                              lambda z = z[doc][word], phi = phi : phi[z]),
                    value = data[doc][word], 
                    observed = True)
                  for doc in range(M) for word in range(N[doc])])

model = pm.Model([theta, phi, z, w])


## Tracing results

- use mcmc to sample from the model 
- show the percentages with which each document corresponds to a certain topic
- show the most relevant words of a certain topic

In [ ]:
mcmc = pm.MCMC(model)
mcmc.sample(40000, 10000, 1)

In [ ]:
# get the traces
phi_tr = [ mcmc.trace('phi_%i' % topic )[:][:, 0] for topic in range(K)]
# plt.hist(phi[:,0])
theta_tr = [ mcmc.trace('theta_%i' % doc)[:][:, 0] for doc in range(M)]
z_tr = [ mcmc.trace('z_%i' % doc )[:] for doc in range(M)]

In [ ]:
print("Procentele cu care fiecare document apartine fiecarui topic : ")
for doc in range(M):
    print('Documentul {} : {}'.format(doc, np.mean(theta_tr[doc], axis = 0).tolist()) )

In [ ]:
wdict_inv = {v : k for k, v in wdict.items()} # reverse mapping from word - index to index - word
no_words = 3 # number of words to show from each topic

print("Cele mai importante {} cuvinte din fiecare topic :".format(no_words))
for topic in range(K):
    most_important_index = np.argsort(phi_tr[topic].mean(axis = 0))[-no_words:]
    most_important_words = [wdict_inv[index] for index in most_important_index ]
    print("Topicul {} : {}".format(topic, most_important_words))